# PGF for LDA

In [1]:
import algopy
from algopy import UTPM

import numpy as np
import numpy.random as rn

from pyaudi import gdual_double as gdual
from pyaudi import exp, log

from scipy.stats import binom, multinomial, nbinom
from scipy.misc import factorial

### Define dummy data

In [2]:
# phi = np.array([[0.1, 0.8, 0.1], [0.5, 0.2, 0.3]]) # distribution over words
# theta = np.array([0.3, 0.7])                       # distribution over topics
# K, V = phi.shape      # K = number of topics, V = size of vocab
# N = 4                 # number of tokens in a document

N = 4   # number of tokens in a document
K = 2   # number of topics
V = 3   # number of unique word types

concen1 = 0.1 # concentration param. for topics.  when 0 < alpha < 1
              # topics are low entropy (i.e., peaked around a single val)
              # when alpha > 1, topics are high entropy
phi = rn.dirichlet(np.ones(V) * concen1, size=K)
assert phi.shape == (K, V)

concen2 = 1.  # concentration param. for document dist over topics
theta = rn.dirichlet(np.ones(K) * concen2)
assert theta.shape == (K,)

### True single-word marginals
Let $Y_v$ be the count of word type $w_v$ in a document, where $v$ is the word index. Compute the true $P(Y_v = y_v)$ for $y_v = 0, 1, .., N$.

In [3]:
p = theta.dot(phi).reshape(V, 1)
x = [range(N + 1) for _ in range(V)]
true_marginals = binom.pmf(x, N, p) # shape = (V, N + 1)

### PGF single-word marginals

In [4]:
def pgf_marginal(v, y_v, phi, theta): # Compute P(Y_v = y_v) = P(count(w_v) = y_v)
    D = y_v + 1
    u_v = UTPM(np.zeros((D, 1)))
    if D > 1:
        u_v.data[1, 0] = 1
        
    u = algopy.ones(V, dtype=u_v)
    u[v] = u_v
    t = phi.dot(u)
    s = theta.dot(t)
    h = np.power(s, N)
    return h.data[:, 0][y_v]

# Oberve 3 tokens of word type w_0
y_v, v = 3, 0
print 'PGF marginal:', pgf_marginal(v, y_v, phi, theta)
print 'True marginal:', true_marginals[0, 3]

PGF marginal: 0.021432136181
True marginal: 0.021432136181


Observe a document of length $N$ with word counts $y = [y_0, ..., y_V]$, where $\sum_{i=0}^V y_i = N$. Find the **single-word** marginal probabilities.

In [5]:
def pgf_marginals(y, phi, theta): # Compute [P(Y_0 = y[0]), ..., P(Y_V = y[V])]
    D = np.max(y) + 1
    u_v = UTPM(np.zeros((D, 1)))
    if D > 1:
        u_v.data[1, :] = 1
    
    u = algopy.ones((V, V), dtype=u_v)
    np.fill_diagonal(u, u_v)
    t = phi.dot(u)
    s = theta.dot(t)
    h = np.power(s, N)
    return [h_v.data[:, 0][y[i]] for i, h_v in enumerate(h)]

# Observe 2 tokens of w_0, 1 token of w_1, and 1 token of w_2
y = np.array([2, 1, 1])
print 'PGF marginals:', pgf_marginals(y, phi, theta)
print 'True marginals:', true_marginals[np.arange(V), y]

PGF marginals: [0.13928347000994659, 0.021433712715387594, 1.9924846367143468e-05]
True marginals: [  1.39283470e-01   2.14337127e-02   1.99248464e-05]


In [6]:
# Observe 1 w_0, 0 w_1, and 3 w_2's
y = np.array([1, 0, 3])
print 'PGF marginals:', pgf_marginals(y, phi, theta)
print 'True marginals:', true_marginals[np.arange(V), y]

PGF marginals: [0.40230106072002236, 0.0012368264207843441, 4.944043330748893e-16]
True marginals: [  4.02301061e-01   1.23682642e-03   4.94404333e-16]


In [7]:
# Observe 0 w_0, 4 w_1's, and 0 w_2
y = np.array([0, 4, 0])
print 'PGF marginals:', pgf_marginals(y, phi, theta)
print 'True marginals:', true_marginals[np.arange(V), y]

PGF marginals: [0.4357466380743994, 0.43573595188145658, 0.99998007500475483]
True marginals: [ 0.43574664  0.43573595  0.99998008]


### PGF joint marginals

In [8]:
def pgf_joint_marginal(v, w, y_v, y_w, phi, theta): # Compute P(Y_v = y_v, Y_w = y_w)
    # Init gdual objects
    order = y_v + y_w
    u_v = gdual(0, "v", order)
    u_w = gdual(0, "w", order)
    
    K, V = phi.shape
    u = [1] * V
    u[v] = u_v
    u[w] = u_w
    
    t = phi.dot(u)
    s = theta.dot(t)
    h = np.power(s, N)
    
    # Evaluate the derivative
    return h.get_derivative([y_v, y_w])/(factorial(y_v) * factorial(y_w))

y_v, v = 1, 0 # 1 count of word 0
y_w, w = 2, 2 # 2 counts of word 2
print 'PGF marginal:', pgf_joint_marginal(v, w, y_v, y_w, phi, theta)
print 'True marginal:', multinomial.pmf([y_v, N-(y_v+y_w), y_w], n=N, p=p.reshape(-1))

PGF marginal: 4.53665451709e-11
True marginal: 4.53665451699e-11


## With growth

Add growth:
- $n = $ number of tokens in the document (observed)
- $m = \sum_{i=1}^n x_i$, where $x_i \sim log(\rho)$
- $\mathbf{y} \sim mult(m, \boldsymbol{\theta}^T \mathbf{\Phi})$

In [9]:
rho = 0.3 # growth parameter
def pgf_marginal_growth(v, y_v, phi, theta, rho): # Compute P(Y_v = y_v) = P(count(w_v) = y_v)
    D = y_v + 1
    u_v = UTPM(np.zeros((D, 1)))
    if D > 1:
        u_v.data[1, 0] = 1
        
    u = algopy.ones(V, dtype=u_v)
    u[v] = u_v
    t = phi.dot(u)
    s = theta.dot(t)
    r = np.log(1 - rho*s) / np.log(1 - rho)
    h = np.power(r, N)
    return h.data[:, 0][y_v]

# Oberve 3 tokens of word type w_0
y_v, v = 3, 0
print 'PGF marginal:', pgf_marginal_growth(v, y_v, phi, theta, rho)
#print 'True marginal:', true_marginals[0, 3]

PGF marginal: 0.0410764887247


In [10]:
def pgf_marginals_growth(y, phi, theta, rho): # Compute [P(Y_0 = y[0]), ..., P(Y_V = y[V])]
    D = np.max(y) + 1
    u_v = UTPM(np.zeros((D, 1)))
    if D > 1:
        u_v.data[1, :] = 1
    
    u = algopy.ones((V, V), dtype=u_v)
    np.fill_diagonal(u, u_v)
    t = phi.dot(u)
    s = theta.dot(t)
    r = np.log(1 - rho*s) / np.log(1 - rho)
    h = np.power(r, N)
    return [h_v.data[:, 0][y[i]] for i, h_v in enumerate(h)]

# Observe 3 tokens of w_0, 1 token of w_1, and 1 token of w_2
y = np.array([3, 1, 1])
print 'PGF marginals:', pgf_marginals_growth(y, phi, theta, rho)

PGF marginals: [0.041076488724737362, 0.012393070063306142, 2.3941059119496452e-05]


## NB LDA

In [11]:
a = 1    # shape parameter of theta_k
b = 0.5  # rate parameter of theta_k
phi_special = np.full((K, V), 1.0/V) # special case where distribution over word types is uniform

### Single-word marginals

In [12]:
def log_pgf(s, p):
    tmp = np.array([log(1 - p*s_k) for s_k in list(s)]) # b/c log(1-p*s) doesn't work
    return tmp / np.log(1 - p)

def pgf_marginal_nb(v, y_v, phi, a, b): # Compute P(Y_v = y_v)
    order = y_v
    K, V = phi.shape
    
    # Init gdual object
    u = [1] * V
    u_v = gdual(0, "v", order)
    u[v] = u_v
    
    # Compute the joint PGF
    t = phi.dot(u)
    s = log_pgf(t, 1.0 / (1+b))
    h = exp(a * (np.sum(s) - K) * np.log(1 + (1.0/b)))
    
    # Evaluate the derivative
    return h.get_derivative([y_v])/factorial(y_v)

def true_marginal_nb(v, y_v, phi, a, b): # only works for K = 2
    phi_v = phi[:, v].reshape((-1, 1))
    p1, p2 = nbinom.pmf([range(y_v + 1)] * 2, a, 1 - (phi_v/(b+phi_v)))
    return np.convolve(p1, p2)[y_v]

y_v, v = 3, 0

# Special case
print 'PGF marginal:', pgf_marginal_nb(v, y_v, phi_special, a, b)
print 'True marginal:', nbinom.pmf(y_v, K*a, 1 - (1.0 / (V*b + 1)))

# General case
print 'PGF marginal:', pgf_marginal_nb(v, y_v, phi, a, b)
print 'True marginal:', true_marginal_nb(v, y_v, phi, a, b)

PGF marginal: 0.09216
True marginal: 0.09216
PGF marginal: 0.107667581816
True marginal: 0.107667581816


### Joint marginals

In [13]:
def pgf_joint_marginal_nb(v, w, y_v, y_w, phi, a, b): # Compute P(Y_v = y_v, Y_w = y_w)
    order = y_v + y_w
    K, V = phi.shape
    
    # Init gdual objects
    u = [1] * V
    u[v] = gdual(0, "v", order)
    u[w] = gdual(0, "w", order)
    
    # Compute the joint PGF
    t = phi.dot(u)
    s = log_pgf(t, 1.0 / (1+b))
    h = exp(a * (np.sum(s) - K) * np.log(1 + (1.0/b)))
    
    # Evaluate the derivative
    return h.get_derivative([y_v, y_w])/(factorial(y_v) * factorial(y_w))

y_v, v = 1, 0 # 1 count of word 0
y_w, w = 2, 2 # 2 counts of word 2
print 'PGF marginal:', pgf_joint_marginal_nb(v, w, y_v, y_w, phi_special, a, b)

PGF marginal: 0.051407151782
